<a href="https://colab.research.google.com/github/Rameshkumar-V/genai-news-research-tool/blob/main/3_vector_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/news_tool/sample_text.csv")
df.shape

(8, 2)

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


Step 1 : Create source embeddings for the text column

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors_embeddings = encoder.encode(df.text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
dim = vectors_embeddings.shape[1]
dim

768

In [17]:
vectors_embeddings[1][1:5]

array([ 0.00980519, -0.03250129,  0.01868204,  0.01770408], dtype=float32)

In [12]:
vectors_embeddings.shape

(8, 768)

**Step 2 : Build a FAISS Index for vectors**

- FAISS (Facebook AI Similarity Search) is an open-source library for similarity search and clustering of vectors.

In [24]:
import faiss

index = faiss.IndexFlatL2(dim)

In [25]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7b65c918bae0> >

Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

NOTE : L2 IS SOME algorithm.

In [26]:

index.add(vectors_embeddings) # Adding our vectors emebeddings to faiss local storage

**Step 4 : Encode search text using same encorder and normalize the output vector**

In [18]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [28]:
import numpy as np
new_vec = np.array(vec).reshape(1,-1)
new_vec.shape

(1, 768)

In [21]:
#faiss.normalize_L2(svec)

**Step 5: Search for similar vector in the FAISS index created**

In [29]:
distances, I = index.search(new_vec, k=2)
distances

array([[1.3844836, 1.4039097]], dtype=float32)

In [30]:
I

array([[3, 2]])

In [32]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [33]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [34]:
search_query

'I want to buy a polo t-shirt'

NOTE:

You can see that the two results from the dataframe are similar to a search_query